In [7]:
%run random_game.py

Double for Away Team! Home 0, Away 0 with 0 outs in the Top of the 1st, with runner(s) on 2nd
Out for Away Team! Home 0, Away 0 with 1 outs in the Top of the 1st, with runner(s) on 2nd
Out for Away Team! Home 0, Away 0 with 2 outs in the Top of the 1st, with runner(s) on 2nd
Out for Home Team! Home 0, Away 0 with 0 outs in the Bottom of the 1st, with no runners on
Out for Home Team! Home 0, Away 0 with 1 outs in the Bottom of the 1st, with no runners on
Single for Home Team! Home 0, Away 0 with 1 outs in the Bottom of the 1st, with runner(s) on 1st
Out for Home Team! Home 0, Away 0 with 2 outs in the Bottom of the 1st, with runner(s) on 1st
Out for Away Team! Home 0, Away 0 with 0 outs in the Top of the 2nd, with no runners on
Out for Away Team! Home 0, Away 0 with 1 outs in the Top of the 2nd, with no runners on
Out for Away Team! Home 0, Away 0 with 2 outs in the Top of the 2nd, with no runners on
Home Run for Away Team! Away 1, Home 0 with 2 outs in the Top of the 2nd, with no runne

In [8]:
%run random_game.py

Single for Away Team! Home 0, Away 0 with 0 outs in the Top of the 1st, with runner(s) on 1st
Out for Away Team! Home 0, Away 0 with 1 outs in the Top of the 1st, with runner(s) on 1st
Home Run for Away Team! Away 2, Home 0 with 1 outs in the Top of the 1st, with no runners on
Out for Away Team! Away 2, Home 0 with 2 outs in the Top of the 1st, with no runners on
Double for Away Team! Away 2, Home 0 with 2 outs in the Top of the 1st, with runner(s) on 2nd
Single for Away Team! Away 2, Home 0 with 2 outs in the Top of the 1st, with runner(s) on 1st 3rd
Out for Home Team! Away 2, Home 0 with 0 outs in the Bottom of the 1st, with no runners on
Out for Home Team! Away 2, Home 0 with 1 outs in the Bottom of the 1st, with no runners on
Triple for Home Team! Away 2, Home 0 with 1 outs in the Bottom of the 1st, with runner(s) on 3rd
Out for Home Team! Away 2, Home 0 with 2 outs in the Bottom of the 1st, with runner(s) on 3rd
Out for Away Team! Away 2, Home 0 with 0 outs in the Top of the 2nd, 

In [9]:
%run random_game.py

Out for Away Team! Home 0, Away 0 with 1 outs in the Top of the 1st, with no runners on
Out for Away Team! Home 0, Away 0 with 2 outs in the Top of the 1st, with no runners on
Out for Home Team! Home 0, Away 0 with 0 outs in the Bottom of the 1st, with no runners on
Out for Home Team! Home 0, Away 0 with 1 outs in the Bottom of the 1st, with no runners on
Single for Home Team! Home 0, Away 0 with 1 outs in the Bottom of the 1st, with runner(s) on 1st
Out for Home Team! Home 0, Away 0 with 2 outs in the Bottom of the 1st, with runner(s) on 1st
Triple for Home Team! Home 1, Away 0 with 2 outs in the Bottom of the 1st, with runner(s) on 1st 3rd
Single for Home Team! Home 2, Away 0 with 2 outs in the Bottom of the 1st, with runner(s) on 1st 2nd 3rd
Out for Away Team! Home 2, Away 0 with 0 outs in the Top of the 2nd, with no runners on
Out for Away Team! Home 2, Away 0 with 1 outs in the Top of the 2nd, with no runners on
Out for Away Team! Home 2, Away 0 with 2 outs in the Top of the 2nd, 